### VectorStoreRetrieverMemory

벡터 스토어에 메모리를 저장하고 호출할 때마다 가장 연관성 있는 상위 K개의 문서를 쿼리.<br>
대화내용의 순서를 명시적으로 추척하지 않는다는 점에서 다른 대부분의 메모리 클래스와 차이점.

In [1]:
from dotenv import load_dotenv

load_dotenv()

from langchain_teddynote import logging

logging.langsmith("VectorStoreRetrieverMemory")

LangSmith 추적을 시작합니다.
[프로젝트명]
VectorStoreRetrieverMemory


In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS
import faiss


embeddings_model = OpenAIEmbeddings()

embeddings_size = 1536
index = faiss.IndexFlatL2(embeddings_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

In [5]:
from langchain.memory import VectorStoreRetrieverMemory


retriever = vectorstore.as_retriever(search_kwargs = {"k": 1})
memory = VectorStoreRetrieverMemory(retriever = retriever)


memory.save_context(
    inputs={
        "human": "안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다."
    },
    outputs={
        "ai": "안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다."
    },
)
memory.save_context(
    inputs={"human": "프로젝트에서 어떤 역할을 맡았나요?"},
    outputs={
        "ai": "제가 맡은 역할은 백엔드 개발자였습니다. 사용자 데이터 처리와 서버 로직 개발을 담당했으며, RESTful API를 구현하여 프론트엔드와의 통신을 담당했습니다. 또한, 데이터베이스 설계에도 참여했습니다."
    },
)
memory.save_context(
    inputs={
        "human": "팀 프로젝트에서 어려움을 겪었던 경험이 있다면 어떻게 해결했나요?"
    },
    outputs={
        "ai": "프로젝트 초기에 의사소통 문제로 몇 가지 어려움이 있었습니다. 이를 해결하기 위해 저희 팀은 정기적인 미팅을 갖고 각자의 진행 상황을 공유했습니다. 또한, 문제가 발생했을 때는 적극적으로 의견을 나누고, 합리적인 해결책을 찾기 위해 노력했습니다."
    },
)
memory.save_context(
    inputs={"human": "개발자로서 자신의 강점은 무엇이라고 생각하나요?"},
    outputs={
        "ai": "제 강점은 빠른 학습 능력과 문제 해결 능력입니다. 새로운 기술이나 도구를 빠르게 습득할 수 있으며, 복잡한 문제에 직면했을 때 창의적인 해결책을 제시할 수 있습니다. 또한, 팀워크를 중시하며 동료들과 협력하는 것을 중요하게 생각합니다."
    },
)

질문을 했을 때 Vector Store로부터 k개의 가장 관련성 높은 대화를 반환.<br>
대화 내용이 매우 많아졌을 때 이전의 대화내용에서 연관성 있는 대화 내용을 가져와 답변을 생성하는데 이용 가능.

In [6]:
print(memory.load_memory_variables({"human": "면접자의 전공은 무엇인가요?"})["history"])

human: 안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다.
ai: 안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다.
